## Overview
**Liquid Clustering**: Databricks' next-generation data clustering feature that automatically manages physical data organization for Delta tables.

### What is Liquid Clustering?
- Liquid Clustering is a **data layout optimization technique** for Delta tables.
- It automatically manages clustering without requiring manual Z-order or partitioning.
- Data is **physically organized** on disk to minimize scan cost for frequently queried columns.
- It provides **better performance** for selective queries and incremental updates.
### Key Points
- Introduced in Databricks Runtime 13.3+
- Works with Delta tables only
- Replaces Manual partitioning and Z-Ordering
- Physically reorders data? Yes, in background compaction and OPTIMIZE runs
- Maintenance Databricks handles clustering maintenance automatically
### Typical Use Cases
- Large tables with frequent inserts, updates, and deletes.
- Query filtering on specific columns like customer_id, region, order_date.
- Scenarios where manual ZORDER tuning is difficult or costly.

In [0]:
%sql
-- Step 1: Create a Delta table using Liquid Clustering
-- The CLUSTER BY clause enables liquid clustering automatically.

CREATE TABLE IF NOT EXISTS sales1_catalog.inputdb.sales_orders_liquid 
(
  order_id INT,
  customer_id INT,
  region STRING,
  product STRING,
  quantity INT,
  price DOUBLE,
  order_date DATE
)
USING DELTA
CLUSTER BY (customer_id, region);

In [0]:
%sql
-- Step 2: Insert sample data (multiple small batches)
-- Each insert simulates separate data ingestion.

INSERT INTO sales1_catalog.inputdb.sales_orders_liquid VALUES
 (1, 101, 'North', 'Laptop', 2, 65000, '2025-10-01'),
 (2, 102, 'South', 'Headphones', 5, 2500, '2025-10-01'),
 (3, 103, 'West', 'Desk Chair', 3, 4500, '2025-10-02');

INSERT INTO sales1_catalog.inputdb.sales_orders_liquid VALUES
 (4, 101, 'North', 'Keyboard', 1, 1200, '2025-10-03'),
 (5, 104, 'East', 'Monitor', 2, 9500, '2025-10-03'),
 (6, 105, 'South', 'Mouse', 4, 700, '2025-10-03');


In [0]:
%sql
-- Step 3: Query data
SELECT * FROM sales1_catalog.inputdb.sales_orders_liquid;

In [0]:
%sql
-- Step 4: View table details
DESCRIBE DETAIL sales1_catalog.inputdb.sales_orders_liquid

In [0]:
%sql
DESCRIBE HISTORY  sales1_catalog.inputdb.sales_orders_liquid;

In [0]:
%sql
-- Step 5: Simulate updates (which trigger reclustering under the hood)
UPDATE sales1_catalog.inputdb.sales_orders_liquid
SET price = price * 1.05
WHERE region = 'North';

In [0]:
%sql
-- Step 6: Verify table history (you will see multiple operations)
DESCRIBE HISTORY sales1_catalog.inputdb.sales_orders_liquid;

In [0]:
%sql
UPDATE sales1_catalog.inputdb.sales_orders_liquid
SET price = price * 1.05
WHERE region = 'South';

In [0]:
%sql
DELETE FROM sales1_catalog.inputdb.sales_orders_liquid
WHERE region = 'East';

In [0]:
%sql
DESCRIBE HISTORY sales1_catalog.inputdb.sales_orders_liquid;